# CONCEITOS ESSENCIAIS

### Porque o Python usa processos e não threads para cálculo paralelo

>O CPython é a implementação oficial da linguagem Python, escrita em C.
O código Python é primeiro convertido em bytecode, uma forma intermédia que o interpretador executa passo a passo.
Para garantir segurança e simplicidade na gestão de memória, o CPython impõe um bloqueio interno — o GIL (Global Interpreter Lock) — que permite que apenas um thread execute bytecode de cada vez dentro do mesmo processo.
Um thread é uma linha de execução dentro de um processo, que partilha a mesma memória.
Como o GIL impede que vários threads executem bytecode simultaneamente, o Python não obtém verdadeiro paralelismo em CPU.
Por isso, para aproveitar vários cores, o paralelismo faz-se com processos independentes, cada um com o seu próprio GIL.
O MPI segue exatamente este modelo: cada processo (rank) tem a sua memória e comunica com os outros.



### Serialização e pickling

>Processos não partilham memória; precisam de enviar dados entre si.
Para isso, os objetos Python são transformados em sequências de bytes — processo chamado serialização.
O formato padrão é o pickle (pickling para converter, unpickling para reconstruir).
Um objeto Python — por exemplo, um dicionário ou lista — contém referências e metadados; o pickle percorre todos os elementos e guarda-os em bytes (0 e 1) para transporte.
É flexível, mas tem custo elevado em CPU e memória.



### send vs Send em mpi4py

>O mpi4py tem duas formas de comunicação:


| Método                                       | Tipo de dados                          | Serialização   | Uso típico                    |
| -------------------------------------------- | -------------------------------------- | -------------- | ----------------------------- |
| `comm.send(obj)` / `comm.recv()`             | Objetos Python completos               | Usa **pickle** | Metadados, mensagens pequenas |
| `comm.Send([buf, MPI.TYPE])` / `comm.Recv()` | **Buffers binários** (`numpy.ndarray`) | **Sem pickle** | Grandes matrizes numéricas    |



    Um objeto Python é uma estrutura de alto nível com referências e atributos.
    Um buffer binário é apenas uma sequência contígua de bytes na memória — como os que o NumPy usa para armazenar matrizes — que pode ser transmitida diretamente, sem codificação adicional.



### Matrizes Python vs NumPy

>Listas de listas (Python):

        - Cada elemento é um objeto separado; os dados não estão contíguos.
        - É preciso usar pickling para enviar, o que é lento.

>numpy.ndarray:

        - Guarda os valores como bytes contínuos e tipados (int32, float64, uint8, …).
        - Pode ser enviado diretamente com comm.Send([A, MPI.INT]), sem pickle, e é processado por código C otimizado.
        

### Em resumo

>O CPython interpreta bytecode e usa o GIL, que limita threads a um de cada vez.

>O paralelismo real em CPU usa processos independentes (como no MPI).

>O pickling converte objetos Python em bytes, mas é lento.

>Send/Recv evitam pickle e usam buffers binários (NumPy).

>NumPy é essencial em HPC: dados contínuos, comunicações rápidas e total compatibilidade com MPI.




        

### CLASSIFICAÇÃO DOS EXEMPLOS



| Exemplo                         | Tipo de paralelismo   | Subtipo / padrão                                      | Descrição resumida                                                                               |
| ------------------------------- | --------------------- | ----------------------------------------------------- | ------------------------------------------------------------------------------------------------ |
| **2.1 – Hello ordenado**        | **Control parallel**  | —                                                     | Demonstra coordenação e sincronização entre processos independentes.                             |
| **2.2 – Broadcast**             | **Data parallel**     | Coletivo / replicação global                          | Um processo (root) gera dados e difunde-os a todos os outros.                                    |
| **2.3 – Scatter/Gather**        | **Data parallel**     | **Espacial (distribuição por blocos)**                | A matriz é dividida no espaço (por linhas); cada processo trabalha num subbloco distinto.        |
| **2.4 – Reduce (soma global)**  | **Data parallel**     | **Espacial (redução coletiva)**                       | Combina resultados locais de diferentes regiões espaciais num único resultado global.            |
| **2.5 – Ponto-a-ponto (anel)**  | **Pipeline parallel** | **Temporal (fluxo sequencial)**                       | Cada processo envia dados ao seguinte e recebe do anterior, formando um fluxo no tempo.          |
| **2.6 – Allgather**             | **Data parallel**     | **Espacial (partilha total)**                         | Todos os processos recebem os blocos de dados de todos os outros.                                |
| **2.A – Áudio (file-per-rank)** | **Data parallel**     | **Independente / por lote (embarrassingly parallel)** | Cada processo analisa ficheiros distintos e autónomos; não há dependência espacial nem temporal. |
| **2.B – Imagem (blur 3×3)**     | **Data parallel**     | **Espacial com halo exchange**                        | Cada processo processa uma região contígua da imagem e troca fronteiras com vizinhos.            |



**Nota**

>**Paralelismo espacial** → divide um mesmo conjunto de dados contínuo (como uma matriz, imagem ou volume) em regiões adjacentes.
Cada processo trata uma parte do espaço (linhas, blocos, pixels vizinhos).
Exemplo: Scatter/Gather e o Blur 3×3.

>**Paralelismo por ficheiro (ou por lote)** → divide um conjunto de tarefas independentes, cada uma completa por si (como vários ficheiros áudio).
Aqui, os dados não formam um espaço contínuo nem há fronteiras partilhadas.
É um caso de data parallel independente ou embarrassingly parallel, também descrito como task-level data parallel.




# 1 — Hello ordenado por rank (barreiras + “turnos”)

In [1]:
%%writefile ex1_hello_ordenado.py
from mpi4py import MPI                     # Importa a interface MPI para Python
comm = MPI.COMM_WORLD                      # Communicator global com todos os processos do job
rank = comm.Get_rank()                     # Identificador (0..size-1) deste processo
size = comm.Get_size()                     # Número total de processos no communicator
host = MPI.Get_processor_name()            # Nome do nó físico onde este processo corre

msg = f"Hello from rank {rank}/{size} on {host}"  # Mensagem única por processo

for r in range(size):                      # Percorre “turnos” r = 0..size-1
    comm.Barrier()                         # Sincroniza todos os processos neste ponto
    if rank == r:                          # Só imprime quem tem o turno (rank == r)
        print(msg, flush=True)             # flush=True para não atrasar a escrita

comm.Barrier()                             # Barreira final para sair em conjunto


Writing ex1_hello_ordenado.py


# 2 — Broadcast: root cria matriz inteira e envia a todos

In [4]:
%%writefile ex2_broadcast_matriz.py
from mpi4py import MPI                     # MPI em Python
import numpy as np                         # NumPy para matrizes
comm = MPI.COMM_WORLD                      # Communicator global
rank = comm.Get_rank()                     # Rank do processo

if rank == 0:                              # Só o root cria os dados
    np.random.seed(42)                     # Semente fixa (reprodutível)
    A = np.random.randint(0, 10,           # Matriz 4x5 de inteiros [0..9]
                          size=(4, 5),
                          dtype=np.int32)
else:
    A = None                               # Nos restantes processos ainda não há A

A = comm.bcast(A, root=0)                  # Root envia A, todos recebem uma cópia
print(f"[rank {rank}] A=\n{A}")            # Mostra a mesma matriz em todos os ranks


Overwriting ex2_broadcast_matriz.py


# 3 — Scatter/Gather por linhas: processar e recompor

In [5]:
%%writefile ex3_scatter_gather_linhas.py
from mpi4py import MPI                     # MPI
import numpy as np                         # NumPy
comm = MPI.COMM_WORLD                      # Communicator
rank = comm.Get_rank()                     # Rank local
size = comm.Get_size()                     # Nº total de processos

rows, cols = 8, 6                          # Dimensão da matriz
if rank == 0:                              # Root gera a matriz inteira
    np.random.seed(0)                      # Reprodutível
    A = np.random.randint(0, 100,          # Inteiros 0..99
                          size=(rows, cols),
                          dtype=np.int32)
    counts = [rows // size] * size         # Linhas base por processo
    for i in range(rows % size):           # Distribui o resto 1 a 1
        counts[i] += 1
    displs = [sum(counts[:i]) for i in range(size)]  # Deslocamentos por processo
else:
    A = None                               # Sem matriz nos não-root
    counts = None                          # Sem contagens ainda
    displs = None                          # Sem deslocamentos ainda

counts = comm.bcast(counts, root=0)        # Todos ficam a conhecer as contagens
displs = comm.bcast(displs, root=0)        # E os deslocamentos

h_local = counts[rank]                     # Nº de linhas do meu bloco
local = np.empty((h_local, cols),          # Buffer local vazio
                 dtype=np.int32)

if rank == 0:                              # Root prepara o Scatterv por elementos
    sendbuf = [A, (np.array(counts)*cols), (np.array(displs)*cols), MPI.INT]
else:
    sendbuf = None                         # Outros não usam sendbuf

comm.Scatterv(sendbuf, local, root=0)      # Distribui os blocos de linhas

local = local + 1                          # “Trabalho” local (ex.: somar 1)

if rank == 0:                              # Root prepara o buffer de recolha
    A_out = np.empty((rows, cols), dtype=np.int32)
    recvbuf = [A_out, (np.array(counts)*cols), (np.array(displs)*cols), MPI.INT]
else:
    A_out = None
    recvbuf = None

comm.Gatherv(local, recvbuf, root=0)       # Recolhe sub-matrizes para o root

if rank == 0:                              # Root mostra antes/depois
    print("A original:\n", A)
    print("A_out = A + 1:\n", A_out)


Writing ex3_scatter_gather_linhas.py


# 4 — Reduce (soma global elemento-a-elemento)

In [6]:
%%writefile ex4_reduce_soma_matrizes.py
from mpi4py import MPI                     # MPI
import numpy as np                         # NumPy
comm = MPI.COMM_WORLD                      # Communicator
rank = comm.Get_rank()                     # Rank
size = comm.Get_size()                     # Nº de processos

np.random.seed(123 + rank)                 # Dados distintos em cada rank
A = np.random.randint(0, 10,               # Matriz 3x4 de inteiros
                      size=(3, 4),
                      dtype=np.int32)

sum_A = comm.reduce(A, op=MPI.SUM, root=0) # Soma ponto-a-ponto entregue no root

print(f"[rank {rank}] A=\n{A}")            # Mostra a sua contribuição
if rank == 0:                              # Root imprime o resultado final
    print("[root] Soma global:\n", sum_A)


Writing ex4_reduce_soma_matrizes.py


# 5 — Ponto-a-ponto (anel): Sendrecv evita deadlock

In [7]:
%%writefile ex5_p2p_anel_sendrecv.py
from mpi4py import MPI                     # MPI
import numpy as np                         # NumPy
comm = MPI.COMM_WORLD                      # Communicator
rank = comm.Get_rank()                     # Rank deste processo
size = comm.Get_size()                     # Nº de processos

np.random.seed(7 + rank)                   # Vetor inteiro distinto por rank
x = np.random.randint(0, 100,              # 5 inteiros 0..99
                      size=5,
                      dtype=np.int32)

src = (rank - 1) % size                    # Vizinhos no anel (cíclico)
dst = (rank + 1) % size                    #   src = esquerda ; dst = direita

y = np.empty_like(x)                       # Buffer de receção com mesma forma/tipo
comm.Sendrecv(sendbuf=x, dest=dst,         # Envia x → vizinho direito
               sendtag=0,                   # Tag do envio
               recvbuf=y, source=src,       # Recebe ← vizinho esquerdo
               recvtag=0)                   # Tag da receção

print(f"[rank {rank}] x local   = {x}")    # Vetor deste processo
print(f"[rank {rank}] y recebido= {y}")    # Vetor recebido do vizinho


Writing ex5_p2p_anel_sendrecv.py


# 6 — Allgather: todos recebem os blocos de todos

In [8]:
%%writefile ex6_allgather_blocos_pequenos.py
from mpi4py import MPI                     # MPI
import numpy as np                         # NumPy
comm = MPI.COMM_WORLD                      # Communicator
rank = comm.Get_rank()                     # Rank
size = comm.Get_size()                     # Nº de processos

np.random.seed(77 + rank)                  # Bloco diferente por rank
B_local = np.random.randint(0, 10,         # “Bloco” 2x3 de inteiros
                            size=(2, 3),
                            dtype=np.int32)

blocks = comm.allgather(B_local)           # Lista de blocos (um por rank) entregue a todos
B = np.vstack(blocks)                      # Empilha verticalmente (2*size x 3)

print(f"[rank {rank}] bloco local:\n{B_local}")  # O meu bloco
print(f"[rank {rank}] empilhado:\n{B}")          # Matriz final (idêntica em todos)


Writing ex6_allgather_blocos_pequenos.py


# A — Áudio (WAV PCM): file-per-rank por género (tipo GTZAN)

>Enquadramento: paralelismo de dados (cada rank trata ficheiros diferentes). Pastas por género: root/blues/*.wav, root/classical/*.wav, … (GTZAN segue isto). Calcula-se RMS por ficheiro e média de RMS por género, agregando no root.

>Classificação: data parallel (file-per-rank), comunicação baixa (difusão da lista + gather).

In [12]:
%%writefile ex7a_audio_file_per_rank_por_genero.py
from mpi4py import MPI                          # Interface MPI para Python
import numpy as np                               # NumPy para vetores/matrizes e cálculos
import os, wave                                  # 'os' para filesystem; 'wave' para WAV PCM
from collections import defaultdict              # Dicionário de listas para acumular por género
import re                                        # 're' para reconhecer padrão de nomes (modo flat)

comm = MPI.COMM_WORLD                            # Communicator global
rank = comm.Get_rank()                           # Identificador deste processo (0..size-1)
size = comm.Get_size()                           # Número total de processos

ROOT_DIR = "/projects/F202500003HPCVLABUTAD/ccvca/intermediate_mpi/audio"  # Pasta com os WAV
os.makedirs(ROOT_DIR, exist_ok=True)             # Garante que a pasta existe (não cria ficheiros)

# Padrão de nomes para o modo "flat": genero.NNNNN.wav (ex.: blues.00042.wav)
pat_flat = re.compile(r"^([a-zA-Z0-9_-]+)\.(\d{5})\.wav$", re.IGNORECASE)

def list_wavs_by_genre(root):
    """Devolve {genero: [caminhos_wav,...]}; suporta subpastas ou pasta única (flat)."""
    mapping = defaultdict(list)                  # Estrutura de saída

    # --- MODO SUBPASTAS: audio/<genero>/*.wav ---
    subdirs = [d for d in sorted(os.listdir(root))
               if os.path.isdir(os.path.join(root, d))]     # Lista de subpastas
    found = 0                                               # Contador de WAVs encontrados
    for g in subdirs:                                       # Para cada género (subpasta)
        gpath = os.path.join(root, g)                       # Caminho completo
        for fn in sorted(os.listdir(gpath)):                # Itera ficheiros na subpasta
            if fn.lower().endswith(".wav"):                 # Só WAV
                mapping[g].append(os.path.join(gpath, fn))  # Acrescenta caminho
                found += 1
    if found > 0:                                           # Se já encontrámos, devolve
        return mapping

    # --- MODO FLAT: audio/genero.NNNNN.wav ---
    for fn in sorted(os.listdir(root)):                     # Itera ficheiros na pasta raiz
        if not fn.lower().endswith(".wav"):                 # Ignora não-WAV
            continue
        m = pat_flat.match(fn)                              # Tenta casar com genero.NNNNN.wav
        if m:
            genero = m.group(1)                             # Grupo 1 = prefixo do género
            mapping[genero].append(os.path.join(root, fn))  # Acrescenta caminho
    return mapping                                          # Pode devolver vazio se nada casar

def wav_rms_int16(path):
    """Lê WAV 8/16-bit PCM mono/estéreo e devolve RMS inteiro (canal 0 se multi-canal)."""
    with wave.open(path, "rb") as wf:                       # Abre WAV para leitura
        nchan = wf.getnchannels()                           # Nº de canais (1=mono, 2=stereo, ...)
        width = wf.getsampwidth()                           # Bytes por amostra (1=8bit, 2=16bit)
        sr = wf.getframerate()                              # Taxa de amostragem (não usada aqui)
        nframes = wf.getnframes()                           # Nº de frames (amostras por canal)
        raw = wf.readframes(nframes)                        # Lê todas as amostras

    if width == 2:                                          # Caso 16-bit PCM (comum no GTZAN)
        data = np.frombuffer(raw, dtype="<i2")              # Converte bytes → int16 little-endian
    elif width == 1:                                        # Caso 8-bit unsigned PCM
        d = np.frombuffer(raw, dtype="uint8")               # Lê como sem sinal
        data = (d.astype(np.int16) - 128)                   # Recentra em 0 (−128..+127)
    else:
        raise ValueError("A demo suporta apenas WAV 8-bit ou 16-bit PCM.")

    if nchan > 1:                                           # Se multi-canal
        data = data[::nchan]                                # Extrai canal 0 (amostras intercaladas)

    rms = int(np.sqrt(np.mean((data.astype(np.int64))**2))) # RMS de amplitude, em contagens PCM
    return rms                                              # Inteiro (ex.: 4817)

# --- ROOT constrói a lista global (género, caminho) ---
if rank == 0:
    by_genre = list_wavs_by_genre(ROOT_DIR)                 # Mapeamento genero → lista de WAVs
    all_pairs = [(g, p) for g, lst in by_genre.items() for p in lst]  # “Flat” de pares
else:
    all_pairs = None                                        # Outros ranks aguardam difusão

all_pairs = comm.bcast(all_pairs, root=0)                   # Todos recebem a lista global
N = len(all_pairs)                                          # Total de ficheiros

# --- Partição quase uniforme da lista global pelos ranks ---
counts = [N // size] * size                                 # Quociente base por rank
for i in range(N % size):                                   # Distribui o resto aos primeiros
    counts[i] += 1
displs = [sum(counts[:i]) for i in range(size)]             # Deslocamentos iniciais por rank
my_pairs = all_pairs[displs[rank]:displs[rank] + counts[rank]]  # Sublista deste rank

# --- Processamento local: RMS por ficheiro e acumular por género ---
local_stats = defaultdict(list)                             # genero → [rms, ...] local
for g, path in my_pairs:                                    # Itera os ficheiros atribuídos
    try:
        rms = wav_rms_int16(path)                           # Calcula RMS em contagens PCM
        local_stats[g].append(int(rms))                     # Guarda como inteiro
    except Exception:
        # Mantém simples: ignora erros de leitura individuais
        pass

# --- Agregação no root: juntar todas as listas por género ---
gathered = comm.gather(local_stats, root=0)                 # Root recolhe dicionários locais

if rank == 0:
    acc = defaultdict(list)                                 # genero → [rms de todos os ranks]
    for d in gathered:                                      # Para cada rank
        for g, lst in d.items():                            # Para cada género
            acc[g].extend(lst)                              # Junta valores

    # Impressão do cabeçalho com duas métricas: RMS (contagens) e dBFS
    print("Genero\tN_fich\tRMS_medio_int\tRMS_medio_dBFS")
    for g in sorted(acc.keys()):
        vals = acc[g]                                       # Lista de RMS por ficheiro
        if not vals:                                        # Se vazia, passa
            continue
        mean_rms = int(round(np.mean(vals)))                # Média em contagens PCM
        if mean_rms > 0:                                    # Converte para dBFS (referência pico 32767)
            dbfs = 20.0 * np.log10(mean_rms / 32767.0)      # 0 dBFS = pico full-scale
            dbfs_str = f"{dbfs:.1f}"                        # Uma casa decimal
        else:
            dbfs_str = "-inf"                               # RMS=0 → -inf dBFS
        print(f"{g}\t{len(vals)}\t{mean_rms}\t{dbfs_str}")  # Linha por género



Overwriting ex7a_audio_file_per_rank_por_genero.py


# B — Imagem: blur 3×3 gaussiano intensificado com halo exchange (grayscale)

>Enquadramento: paralelismo de dados por linhas contíguas.
Cada rank recebe um bloco de linhas da imagem (grayscale uint8), troca uma linha de halo com os vizinhos (topo/base) e aplica iterativamente um filtro 3×3 gaussiano inteiro (média ponderada com pesos 1-2-1) para intensificar o desfoque.
O halo exchange garante continuidade entre blocos e elimina descontinuidades nas fronteiras internas.
Entrada: ficheiro de imagem real (PNG/JPG/TIFF, …).
Leitura: feita com Pillow, que converte a imagem para escala de cinzentos 8-bit.
Saída: imagem processada (imagem_blur.png), gravada com matplotlib em modo grayscale.

>Parâmetros ajustáveis: o número de iterações do blur (REPEATS) controla a intensidade; mais iterações → desfoque mais forte.

>Classificação: data parallel com troca de halos (vizinhança 3×3); comunicação moderada; processamento iterativo local.



In [28]:
%%writefile ex7b_imagem_blur3x3_halos.py    

from mpi4py import MPI                       # Importa a interface MPI para Python (comunicação entre processos).
import numpy as np                            # NumPy: arrays eficientes e operações vectorizadas.
import os                                     # Funções de sistema de ficheiros (verificar diretórios/ficheiros, construir paths).

comm = MPI.COMM_WORLD                         # Communicator com todos os processos do job MPI.
rank = comm.Get_rank()                        # Índice (ID) deste processo no communicator.
size = comm.Get_Size() if hasattr(comm,'Get_Size') else comm.Get_size()  # Nº total de processos (compatível com versões).

IMG_PATH = "/projects/F202500003HPCVLABUTAD/ccvca/intermediate_mpi/imagens"  # Pasta ou ficheiro de entrada.

# -------------------------- I/O --------------------------

def read_image_gray_uint8(path):
    """Lê imagem e converte para grayscale 8-bit [0..255]."""
    from PIL import Image                     # Import local: evita carregar Pillow em todos os nós à cabeça.
    im = Image.open(path).convert("L")        # Abre e força modo 'L' (grayscale 8-bit).
    return np.array(im, dtype=np.uint8)       # Converte para matriz NumPy uint8 (H×W).

def write_image_uint8(path, arr_uint8):
    """Escreve matriz uint8 como imagem grayscale."""
    import matplotlib.image as mpimg          # Import local para salvar imagem sem depender de cv2/scipy.
    mpimg.imsave(path, arr_uint8, cmap='gray', vmin=0, vmax=255)  # Garante escala correta 0–255.

def resolve_image_path(path_or_dir):
    """Se for diretoria, escolhe 1.º ficheiro de imagem; se for ficheiro, valida."""
    exts = (".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff")  # Extensões suportadas.
    if os.path.isdir(path_or_dir):            # Caso de diretoria…
        files = sorted([f for f in os.listdir(path_or_dir)
                        if f.lower().endswith(exts)])          # Filtra e ordena alfabeticamente.
        if not files:                         # Proteção: nenhuma imagem encontrada.
            raise RuntimeError(f"Nenhuma imagem encontrada em {path_or_dir}")
        return os.path.join(path_or_dir, files[0])             # Usa o 1.º ficheiro.
    else:
        if not os.path.isfile(path_or_dir):   # Caso seja caminho inválido.
            raise RuntimeError(f"Ficheiro não existe: {path_or_dir}")
        return path_or_dir                    # É um ficheiro válido: retorna tal como está.

# -------------------------- Root lê e define partição --------------------------

if rank == 0:                                  # Apenas o root acede ao disco.
    chosen = resolve_image_path(IMG_PATH)      # Resolve se é pasta ou ficheiro e devolve o caminho final.
    print(f"[root] Imagem selecionada: {chosen}")
    img = read_image_gray_uint8(chosen)        # Lê a imagem como matriz uint8.
    H, W = img.shape                           # Guarda altura (linhas) e largura (colunas).
    counts = [H // size] * size                # Linhas base por rank (divisão inteira).
    for i in range(H % size):                  # Distribui o resto (as primeiras 'r' ranks recebem +1 linha).
        counts[i] += 1
    displs = [sum(counts[:i]) for i in range(size)]  # Deslocamentos (linha inicial) por rank.
else:
    chosen = img = None                        # Nos outros ranks não há dados ainda.
    H = W = None
    counts = displs = None

# Broadcast de metadados para todos os processos.
chosen = comm.bcast(chosen, root=0)            # Todos passam a conhecer o ficheiro escolhido.
H = comm.bcast(H, root=0)                      # Altura da imagem.
W = comm.bcast(W, root=0)                      # Largura da imagem.
counts = comm.bcast(counts, root=0)            # Nº de linhas atribuídas a cada rank.
displs = comm.bcast(displs, root=0)            # Linha inicial de cada rank.

# -------------------------- Scatter (linhas contíguas) --------------------------

h_local = counts[rank]                          # Nº de linhas do bloco deste rank.
local = np.empty((h_local, W), dtype=np.uint8)  # Buffer para receber o bloco (sem halo).

if rank == 0:
    # Para Scatterv, define-se o buffer de origem como (dados, counts_em_elementos, displs_em_elementos, tipoMPI).
    send_counts = (np.array(counts) * W)        # Cada linha tem W elementos → converter linhas em elementos.
    send_displs = (np.array(displs) * W)        # Deslocamentos também medidos em elementos.
    sendbuf = [img, send_counts, send_displs, MPI.UNSIGNED_CHAR]  # Tipo condiz com uint8.
else:
    sendbuf = None

comm.Scatterv(sendbuf, local, root=0)           # Envia para cada rank o seu bloco de linhas.

# Guarda o bloco original desta execução (útil para blend opcional no fim).
original_local = local.copy()

# -------------------------- Iterações de blur com halo --------------------------

REPEATS = 4                                     # Nº de iterações do filtro (mais iterações → mais desfoque).

for _ in range(REPEATS):                        # Loop de processamento local iterativo.

    # --- Determina vizinhos para troca de halos (sem contorno circular) ---
    up_rank = rank - 1 if rank > 0 else None    # Vizinho de cima; None se este rank é o 0.
    dn_rank = rank + 1 if rank < size - 1 else None  # Vizinho de baixo; None no último rank.

    top_halo = np.empty((1, W), dtype=np.uint8) # Linhas de halo a receber/enviar (topo).
    bot_halo = np.empty((1, W), dtype=np.uint8) # Linhas de halo (base).

    # Pré-preenche halos com “clamping” (replica a borda local) para o caso de não haver vizinho.
    top_halo[...] = local[:1, :]                # Se não houver vizinho de cima, mantém a 1.ª linha.
    bot_halo[...] = local[-1:, :]               # Se não houver vizinho de baixo, mantém a última linha.

    # Troca efetiva de linhas de fronteira com os vizinhos existentes (Sendrecv evita deadlock).
    if up_rank is not None:
        comm.Sendrecv(sendbuf=local[:1, :], dest=up_rank,
                      recvbuf=top_halo,       source=up_rank)
    if dn_rank is not None:
        comm.Sendrecv(sendbuf=local[-1:, :], dest=dn_rank,
                      recvbuf=bot_halo,       source=dn_rank)

    # Constrói um bloco aumentado com 1 linha extra no topo e na base (para aplicar kernel 3×3).
    aug = np.vstack([top_halo, local, bot_halo])

    # --- Filtro Gaussiano 3×3 separável (pesos 1-2-1) sem overflow ---
    acc = np.zeros_like(local, dtype=np.int32)  # Acumulador em 32-bit para evitar saturações.

    # Varre deslocamentos verticais (-1, 0, +1) com pesos 1, 2, 1.
    for di, wv in [(-1, 1), (0, 2), (1, 1)]:
        # Seleciona a faixa vertical alinhada ao bloco local, dentro do bloco aumentado.
        src = aug[1 + di : 1 + di + h_local, :]

        # *** Passo crítico: promover para inteiro largo ANTES de somar/pesar, evitando overflow em uint8. ***
        src32 = src.astype(np.int32, copy=False)

        # “Shifts” horizontais com replicação de borda (coluna fantasma) — tudo em int32.
        left  = np.hstack([src32[:, :1], src32[:, :-1]])   # Vizinhança à esquerda.
        mid   = src32                                      # Coluna central.
        right = np.hstack([src32[:, 1:], src32[:, -1:]])   # Vizinhança à direita.

        # Soma ponderada horizontal (1*left + 2*mid + 1*right) e acumula com o peso vertical wv.
        acc += wv * (left + 2*mid + right)

    # Normalização do kernel gaussiano 3×3: soma de pesos = 16 → dividir por 16 (com arredondamento).
    blur = ((acc + 8) // 16).astype(np.int32)              # “+8” implementa arredondamento por excesso/2.
    blur = np.clip(blur, 0, 255).astype(np.uint8)          # Regressa a uint8, garantindo limites válidos.

    local = blur                                           # Atualiza o bloco para a próxima iteração.

# -------------------------- Blend opcional (recuperar micro-contraste) --------------------------
# BETA = 0.35                                             # Fração do original a misturar (0 → só blur).
# local = np.clip((1.0 - BETA)*local + BETA*original_local, 0, 255).astype(np.uint8)
# Observação: este blend usa 'original_local' GUARDADO no início desta execução, não ficheiros antigos.

# -------------------------- Gather (reconstruir imagem completa) --------------------------

if rank == 0:
    out = np.empty((H, W), dtype=np.uint8)                 # Buffer final no root.
    recv_counts = (np.array(counts) * W)                   # Contagens em elementos.
    recv_displs = (np.array(displs) * W)                   # Deslocamentos em elementos.
    recvbuf = [out, recv_counts, recv_displs, MPI.UNSIGNED_CHAR]
else:
    out = None
    recvbuf = None

comm.Gatherv(local, recvbuf, root=0)                       # Recolhe blocos de todos os ranks para 'out'.

# -------------------------- Guardar resultado --------------------------

if rank == 0:
    out_dir = os.path.dirname(chosen)                      # Usa a mesma pasta da imagem de entrada.
    out_path = os.path.join(out_dir, f"imagem_blur_R{REPEATS}.png")  # Nome inclui nº de iterações.
    write_image_uint8(out_path, out)                       # Grava a imagem processada.
    print(f"[root] Imagem guardada em {out_path}")         # Mensagem final de confirmação.


Overwriting ex7b_imagem_blur3x3_halos.py
